In [37]:
import pandas as pd;
import json;
import math;

In [38]:
map_json = open('datamap.json');
nav = json.load(map_json)['raw'];

dataset = nav['files'];
pd.DataFrame(dataset).reindex(columns=['file','range'])

,file,range
0,SPX_WSJ.csv,"{'start': '2/1/1991', 'end': '3/11/2021'}"
1,SENSEX_WSJ.csv,"{'start': '1/1/1991', 'end': '3/11/2021'}"
2,N225_WSJ.csv,"{'start': '4/1/1991', 'end': '2/11/2021'}"
3,UKX_WSJ.csv,"{'start': '2/1/1991', 'end': '3/11/2021'}"
4,BTC-USD_Yahoo.csv,"{'start': '17/9/2014', 'end': '4/11/2021'}"


In [39]:
stocks={}
names=['data_snp5', 'data_sens', 'data_n225', 'data_ftse'];

for i in names:
    stocks[i] = pd.read_csv('clean/'+i+'.csv').set_index(['Date']);
    print(stocks[i].head());

             Close
Date              
1991-01-04  321.00
1991-01-05  321.00
1991-01-06  321.00
1991-01-07  315.44
1991-01-08  314.90
              Close
Date               
1991-01-04   982.35
1991-01-05   982.35
1991-01-06   982.35
1991-01-07  1010.77
1991-01-08  1010.77
               Close
Date                
1991-01-04  24069.18
1991-01-05  24069.18
1991-01-06  24069.18
1991-01-07  23736.57
1991-01-08  22897.84
             Close
Date              
1991-01-04  2126.1
1991-01-05  2126.1
1991-01-06  2126.1
1991-01-07  2113.3
1991-01-08  2099.9


In [40]:
for p in stocks:
    index=stocks[p]; 
    close=index['Close'];
    rets=[0 for i in range(len(close))];
    #rets2=[0 for i in range(len(close))];
    for i in range(len(close)-1):
        rets[i+1]=math.log10(close[i+1]) - math.log10(close[i]);
        #rets2[i+1]= rets[i+1]*rets[i+1];
    stocks[p]['R']=rets;
    #stocks[p]['R^2']=rets2;
    print(p+"\n", stocks[p].head());

data_snp5
              Close         R
Date                        
1991-01-04  321.00  0.000000
1991-01-05  321.00  0.000000
1991-01-06  321.00  0.000000
1991-01-07  315.44 -0.007588
1991-01-08  314.90 -0.000744
data_sens
               Close         R
Date                         
1991-01-04   982.35  0.000000
1991-01-05   982.35  0.000000
1991-01-06   982.35  0.000000
1991-01-07  1010.77  0.012386
1991-01-08  1010.77  0.000000
data_n225
                Close         R
Date                          
1991-01-04  24069.18  0.000000
1991-01-05  24069.18  0.000000
1991-01-06  24069.18  0.000000
1991-01-07  23736.57 -0.006043
1991-01-08  22897.84 -0.015623
data_ftse
              Close         R
Date                        
1991-01-04  2126.1  0.000000
1991-01-05  2126.1  0.000000
1991-01-06  2126.1  0.000000
1991-01-07  2113.3 -0.002623
1991-01-08  2099.9 -0.002763


In [42]:
returns = pd.concat(
    [stocks[p].drop(columns=['Close']) for p in stocks], 
    axis=1, 
    keys=[i for i in names]);
returns.columns=names;
print(returns.head());

closes = pd.concat(
    [stocks[p].drop(columns=['R']) for p in stocks], 
    axis=1, 
    keys=[i for i in names]);
closes.columns=names;
print(closes.head());

            data_snp5  data_sens  data_n225  data_ftse
Date                                                  
1991-01-04   0.000000   0.000000   0.000000   0.000000
1991-01-05   0.000000   0.000000   0.000000   0.000000
1991-01-06   0.000000   0.000000   0.000000   0.000000
1991-01-07  -0.007588   0.012386  -0.006043  -0.002623
1991-01-08  -0.000744   0.000000  -0.015623  -0.002763
            data_snp5  data_sens  data_n225  data_ftse
Date                                                  
1991-01-04     321.00     982.35   24069.18     2126.1
1991-01-05     321.00     982.35   24069.18     2126.1
1991-01-06     321.00     982.35   24069.18     2126.1
1991-01-07     315.44    1010.77   23736.57     2113.3
1991-01-08     314.90    1010.77   22897.84     2099.9


In [43]:
returns_corr = returns.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)

In [44]:
returns_corr = returns.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1);
closes_corr = closes.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1);

print("Returns\n",returns_corr,"\n\nCloses\n", closes_corr)

Returns
            data_snp5  data_sens  data_n225  data_ftse
data_snp5   1.000000   0.922258   0.904999   0.959394
data_sens   0.922258   1.000000   0.881563   0.926055
data_n225   0.904999   0.881563   1.000000   0.913269
data_ftse   0.959394   0.926055   0.913269   1.000000 

Closes
            data_snp5  data_sens  data_n225  data_ftse
data_snp5   1.000000   0.996090   0.995616   0.993135
data_sens   0.996090   1.000000   0.998799   0.996341
data_n225   0.995616   0.998799   1.000000   0.995860
data_ftse   0.993135   0.996341   0.995860   1.000000
